In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import requests
import json
from datetime import datetime

In [2]:
current_path = os.getcwd()
os.chdir(current_path)

# 1. Lectura de Datos

In [3]:
tarjet_movie_df_0 = pd.read_csv('Data/IMDB_Top250Engmovies2_OMDB_Detailed.csv').iloc[:, 1:]
tarjet_movie_df_0.head(2)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True


## 1.1 Actualización de Datos

In [4]:
# Leer archivo con credenciales API

from tkinter import *
from tkinter import filedialog
from tkinter import Tk

root = Tk()
root.withdraw()
root.wm_attributes('-topmost', 1)

filepath = filedialog.askopenfilename(parent=root, initialdir=current_path)


file = open(filepath, 'r')
credentials = json.load(file)

In [5]:
# Generación de función de Consulta a API IMDB

def movie_info(ID):
    
    if type('a').__name__ != 'str':
        return('Error: Solo cadenas de texto.')
    
    if ID[:2] != 'tt':
        
        url = "https://online-movie-database.p.rapidapi.com/auto-complete"
        
        querystring = {"q": ID}
        
        headers = {
            "X-RapidAPI-Key": credentials['API_KEY']
            ,"X-RapidAPI-Host": credentials['HOST']
        }
        
        response = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)
        
        basic_info = pd.DataFrame()
        
        for i in range(len(response['d'])):
            basic_info = basic_info.append({'id':response['d'][i].get('id')
                                            ,'title':response['d'][i].get('l')
                                            ,'year':response['d'][i].get('y')
                                            ,'type':response['d'][i].get('qid')
                                            ,'rank':response['d'][i].get('rank')
                                            ,'cast':response['d'][i].get('s')
                                            ,'image':response['d'][i].get('i', {}).get('imageUrl')
                                           }
                                           , ignore_index=True
                                          )
        
        basic_info = basic_info[basic_info['type']=='movie']
        basic_info = basic_info[:1]
        
        complete_info = movie_info(basic_info['id'][0])
    
    else:
        
        url_meta = "https://online-movie-database.p.rapidapi.com/title/get-meta-data"
        url_credits = "https://online-movie-database.p.rapidapi.com/title/get-full-credits"
        url_versions = "https://online-movie-database.p.rapidapi.com/title/get-versions"
        url_plot = "https://online-movie-database.p.rapidapi.com/title/get-plots"
        
        querystring_meta = {"ids": ID
                            ,"region":"US"
                           }
        querystring_credits = {"tconst": ID}
        querystring_versions = {"tconst": ID}
        querystring_plot = {"tconst": ID}
        
        headers = {
            "X-RapidAPI-Key": credentials['API_KEY']
            ,"X-RapidAPI-Host": credentials['HOST']
        }
        
        response_meta = json.loads(requests.request("GET", url_meta, headers=headers, params=querystring_meta).text)
        response_credits = json.loads(requests.request("GET", url_credits, headers=headers, params=querystring_credits).text)
        response_versions = json.loads(requests.request("GET", url_versions, headers=headers, params=querystring_versions).text)
        response_plot = json.loads(requests.request("GET", url_plot, headers=headers, params=querystring_plot).text)
        
        complete_info = pd.DataFrame()
        
        complete_info = complete_info.append({'id':ID
                                              ,'title':response_meta.get(ID, {}).get('title', {}).get('title')
                                              ,'year':response_meta.get(ID, {}).get('title', {}).get('year')
                                              ,'rated':response_meta.get(ID, {}).get('certificate')
                                              ,'released':datetime.strptime((response_meta.get(ID, {}).get('releaseDate')+'-01')[:10], '%Y-%m-%d')
                                              ,'runtime':response_meta.get(ID, {}).get('title', {}).get('runtime_minutes')
                                              ,'genre':response_meta.get(ID, {}).get('genres')
                                              ,'directors':[response_credits.get('crew', {}).get('director', {})[k].get('name') for k in range(len(response_credits.get('crew', {}).get('director', {})))]
                                              ,'cinematographer':[response_credits.get('crew', {}).get('cinematographer', {})[k].get('name') for k in range(len(response_credits.get('crew', {}).get('cinematographer', {})))]
                                              ,'writers':[response_credits.get('crew', {}).get('writer', {})[k].get('name') for k in range(len(response_credits.get('crew', {}).get('writer', {})))]
                                              ,'actors':[response_credits.get('cast', {})[k].get('name') for k in range(len(response_credits.get('cast', {})))]
                                              ,'colorations':next(iter(response_versions.get('colorations')), None)
                                              ,'plot':next(iter(response_plot.get('plots')), None).get('text')
                                              ,'language':next(iter(response_versions.get('spokenLanguages')), None)
                                              ,'country':next(iter(response_versions.get('origins')), None)
                                              ,'poster':response_meta.get(ID, {}).get('title', {}).get('image', {}).get('url')
                                              ,'imbd_rating':response_meta.get(ID, {}).get('ratings', {}).get('rating')
                                              ,'imdb_count':response_meta.get(ID, {}).get('ratings', {}).get('ratingCount')
                                              ,'metascore':response_meta.get(ID, {}).get('metacritic', {}).get('metaScore')
                                              ,'meta_count':response_meta.get(ID, {}).get('metacritic', {}).get('reviewCount')
                                             }
                                             , ignore_index=True)
        
    return(complete_info)

In [6]:
# Actualizar datos 250 Películas

tarjet_movie_df = pd.DataFrame()

for i in list(tarjet_movie_df_0['imdbID']):
    tarjet_movie_df = tarjet_movie_df.append(movie_info(i))

tarjet_movie_df.to_csv('Data/Top250Movies.csv')

In [7]:
tarjet_movie_df = pd.read_csv('Data/Top250Movies.csv').iloc[:, 1:]
tarjet_movie_df.head(5)

,id,title,year,rated,released,runtime,genre,directors,cinematographer,writers,actors,colorations,plot,language,country,poster,imbd_rating,imdb_count,metascore,meta_count
0,tt0111161,The Shawshank Redemption,1994.0,R,1994-10-14,NaN,['Drama'],['Frank Darabont'],['Roger Deakins'],"['Stephen King', 'Frank Darabont']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",color,Two imprisoned men bond over a number of years...,en,US,https://m.media-amazon.com/images/M/MV5BMDFkYT...,9.3,2663893.0,81.0,21.0
1,tt0068646,The Godfather,1972.0,R,1972-03-24,NaN,"['Crime', 'Drama']",['Francis Ford Coppola'],['Gordon Willis'],"['Mario Puzo', 'Francis Ford Coppola', 'Mario ...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",color,The aging patriarch of an organized crime dyna...,en,US,https://m.media-amazon.com/images/M/MV5BM2MyNj...,9.2,1846042.0,100.0,16.0
2,tt0071562,The Godfather Part II,1974.0,R,1974-12-18,NaN,"['Crime', 'Drama']",['Francis Ford Coppola'],['Gordon Willis'],"['Francis Ford Coppola', 'Mario Puzo', 'Mario ...","['Al Pacino', 'Robert Duvall', 'Diane Keaton',...",color,The early life and career of Vito Corleone in ...,en,US,https://m.media-amazon.com/images/M/MV5BMWMwMG...,9.0,1264449.0,90.0,18.0
3,tt0468569,The Dark Knight,2008.0,PG-13,2008-07-18,NaN,"['Action', 'Crime', 'Drama', 'Thriller']",['Christopher Nolan'],['Wally Pfister'],"['Jonathan Nolan', 'Christopher Nolan', 'Chris...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",color,When the menace known as the Joker wreaks havo...,en,US,https://m.media-amazon.com/images/M/MV5BMTMxNT...,9.0,2636861.0,84.0,39.0
4,tt0050083,12 Angry Men,1957.0,Approved,1957-04-10,NaN,"['Crime', 'Drama']",['Sidney Lumet'],['Boris Kaufman'],"['Reginald Rose', 'Reginald Rose']","['Martin Balsam', 'John Fiedler', 'Lee J. Cobb...",monochrome,The jury in a New York City murder trial is fr...,en,US,https://m.media-amazon.com/images/M/MV5BMWU4N2...,9.0,786679.0,96.0,10.0


In [ ]:
tarjet_movie_df.info()

In [ ]:
# Revision variables nulas

null_var = tarjet_movie_df.isnull().mean().sort_values(ascending=False)
null_var

In [ ]:
# Se eliminan variables con una gran proporción de nulos (mayor al 50%)

tarjet_movie_df = tarjet_movie_df.drop(columns = list(null_var[null_var >= 0.5].index))
tarjet_movie_df.isnull().mean().sort_values(ascending=False)

In [ ]:
df = pd.read_csv('Data/data.tsv', sep = '	')
df[df['titleType']=='movie']

In [ ]:
df[(df['titleType']=='movie')
   & (df['primaryTitle'].str.contains('Lyon'))]